# ONE REST queries

While the basic `one.search` command allows us to filter experimental sessions by date, lab,
subject etc., you may quickly find that you want to apply more complex queries to restrict your
search output. This can be achieved using the `one.alyx.rest` command. This has access to a range
of tables stored in the database and gives us more flexibility when forming our queries.

<div class="alert alert-info">
Note.

REST queries can only be made while ONE is in online mode.
</div>

## `one.search` vs `one.alyx.rest`
We will get started by importing ONE

In [1]:
from one.api import ONE
one = ONE(base_url='https://openalyx.internationalbrainlab.org')

We can list the possible tables that we have access to, by typing the following command

In [2]:
one.alyx.list_endpoints()

['brain-regions',
 'cache',
 'cache.zip',
 'channels',
 'data-formats',
 'data-repository',
 'data-repository-type',
 'dataset-types',
 'datasets',
 'downloads',
 'files',
 'insertions',
 'labs',
 'locations',
 'new-download',
 'notes',
 'projects',
 'register-file',
 'revisions',
 'sessions',
 'subjects',
 'surgeries',
 'sync-file-status',
 'tags',
 'tasks',
 'trajectories',
 'uploaded',
 'users',
 'water-administrations',
 'water-requirement',
 'water-restricted-subjects',
 'water-restriction',
 'water-type',
 'weighings']

Let's first look at `sessions` table. The information about what filters can be applied to this
table are available under FILTERS
[here](https://openalyx.internationalbrainlab.org/docs/#sessions-list). Let's apply a filter based
on `performance_qte` and `task_protocol`. We can do this using the following expression,

In [3]:
sess_info = one.alyx.rest('sessions', 'list', performance_gte=70, task_protocol='ephys')
sess_info[0]

{'subject': 'SWC_043',
 'start_time': '2020-09-21T19:02:16.707541',
 'number': 1,
 'lab': 'hoferlab',
 'project': 'ibl_neuropixel_brainwide_01',
 'url': 'https://openalyx.internationalbrainlab.org/sessions/4ecb5d24-f5cc-402c-be28-9d0f7cb14b3a',
 'task_protocol': '_iblrig_tasks_ephysChoiceWorld6.4.2'}

Notice how this command doesn't just return the session eID but a dictionary containing information
about each session. We can extract the set of eIDs using the `to_eid` method:

In [4]:
one.to_eid(sess_info)

['4ecb5d24-f5cc-402c-be28-9d0f7cb14b3a',
 'c7bd79c9-c47e-4ea5-aea3-74dda991b48e',
 '4b7fbad4-f6de-43b4-9b15-c7c7ef44db4b',
 'aad23144-0e52-4eac-80c5-c4ee2decb198']

You might have noticed that this same query could have been achieved using the `one.search` method
with the [remote query flag](../one_modes/one_modes.html),

In [5]:
eids = one.search(performance_gte=70, task_protocol='ephys', query_type='remote')
eids

## Accessing other Alyx tables

With the `one.alyx.rest` command we are not only limited to the `sessions` table, but can formulate
queries based on other tables to find session eIDs of interest. Consider the case where we want to
find all sessions that have probe insertions that target a specific ML and AP coordinate. For this
we can formulate our query based on the
[trajectories table](https://openalyx.internationalbrainlab.org/docs/#trajectories-list).
Let's see if there are any probe insertions at the coordinates ML(x) = -2225, and AP(y) = -1894
from bregma.

In [6]:
trajs = one.alyx.rest('trajectories', 'list', x=-2224.97344914846, y=-1893.87527870804)
trajs[0]

{'id': '4c9f8b8c-16c5-45fe-8f5c-3c490ab410c0',
 'probe_insertion': 'b749446c-18e3-4987-820a-50649ab0f826',
 'x': -2224.97344914846,
 'y': -1893.87527870804,
 'z': -118.0,
 'depth': 4056.23152523877,
 'theta': 10.0389727803095,
 'phi': 147.057669457309,
 'roll': 0.0,
 'provenance': 'Ephys aligned histology track',
 'session': {'subject': 'KS023',
  'start_time': '2019-12-10T16:44:12.945000',
  'number': 1,
  'lab': 'cortexlab',
  'id': 'aad23144-0e52-4eac-80c5-c4ee2decb198',
  'task_protocol': '_iblrig_tasks_ephysChoiceWorld6.1.3'},
 'probe_name': 'probe01',
 'coordinate_system': 'IBL-Allen',
 'datetime': '2020-10-22T17:12:18.051432',
 'json': {'2020-06-12T00:39:15_nate': [[-1.0019317591838977,
    0.00034058267716535555,
    0.0016527874015748033,
    0.0024480629921259846,
    0.002829795275590551,
    0.003028614173228347,
    0.0034103464566929134,
    1.0047060407069595],
   [-1.0406470176854647,
    0.00048822751056065705,
    0.0018277480314961014,
    0.0025676767397318587,
    

We can find the session eID associated with this trajectory by looking at the `id` of the `session`
field in the returned dictionary

In [7]:
eid = trajs[0]['session']['id']
eid

'aad23144-0e52-4eac-80c5-c4ee2decb198'

<div class="alert alert-info">

Note.

It is not just sessions that have unique IDs associated with them! Every object stored in Alyx has
a unique UUID, whether it is a trajectory, a subject, a user or a dataset. For example in the above
example we can access the unique ID of the trajectory by typing `traj_id = trajs[0]['id']`


</div>

## Searching with `one.alyx.rest`
The `one.alyx.rest` command is also provides an alternative method to `one.list` for searching
the database for possible keywords that you can use to restrict your queries. For example, if we
 want to find the names of female subjects in the Witten lab that are alive, we can use the
 [subjects table](https://openalyx.internationalbrainlab.org/docs/#subjects-list) to write,

In [8]:
subj_info = one.alyx.rest('subjects', 'list', lab='cortexlab', sex='F', alive=False)
subj_nickname = [subj['nickname'] for subj in subj_info]
subj_nickname

['KS023']